<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/BanglaSER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
%%capture
!pip install gdown

In [6]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [7]:
%%capture
!gdown --id 1fZoWnL0neesTfj3l1kQRNN5O0Sfm3Bjl
time.sleep(5)

!unzip /content/BanglaSER.zip

In [8]:
import os
import librosa
import json
import glob
from sklearn.model_selection import train_test_split

# Emotion mapping based on your dataset description
emotion_map = {
    1: "Happy",
    2: "Sad",
    3: "Angry",
    4: "Surprise",
    5: "Neutral"
}

def parse_filename(filename):
    """Parses the filename to extract the emotion label."""
    parts = filename.split('-')
    if len(parts) >= 3:  # Adjust based on your filename structure
        emotion_label = int(parts[2])  # Assumes the 3rd part is the emotion label
        return emotion_map.get(emotion_label, "Unknown")
    return None

def calculate_audio_length(file_path):
    """Calculates the length of an audio file in seconds."""
    audio, sr = librosa.load(file_path, sr=None)
    return librosa.get_duration(y=audio, sr=sr)

def create_filtered_data(directory):
    """Creates a minimal JSON data structure for audio files in a directory, excluding 'Surprise' emotion."""
    json_data = []

    # Using glob to find all WAV files in the directory
    audio_files = glob.glob(os.path.join(directory, '**/*.wav'), recursive=True)

    for file_path in audio_files:
        filename = os.path.basename(file_path)
        audio_id = os.path.splitext(filename)[0]
        emotion_label = parse_filename(filename)

        # Exclude files with emotion "Surprise"
        if emotion_label and emotion_label != "Surprise":
            audio_length = calculate_audio_length(file_path)

            # Create minimal data entry
            data_entry = {
                "id": audio_id,
                "emotion": emotion_label,
                "length": audio_length,
                "wav": file_path
            }
            json_data.append(data_entry)

    return json_data

def split_and_save_data(json_data, seed=32):
    """Splits the data into train and test sets and saves them to separate JSON files."""
    train, test = train_test_split(json_data, test_size=0.2, random_state=seed)

    # Save train data
    with open('train_data.json', 'w') as train_file:
        json.dump(train, train_file, indent=4)

    # Save test data
    with open('test_data.json', 'w') as test_file:
        json.dump(test, test_file, indent=4)



In [9]:
# Directory path for your audio files
directory_path = '/content/t9h6p943xy-5/BanglaSER'

# Step 1: Create filtered data directly
print("Creating filtered data...")
filtered_data = create_filtered_data(directory_path)

# Step 2: Split and save data to train and test JSON files
print("Splitting and saving data into train and test JSON files...")
split_and_save_data(filtered_data)

print("Train and test JSON files created successfully.")


Creating filtered data...
Splitting and saving data into train and test JSON files...
Train and test JSON files created successfully.


In [11]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/t9h6p943xy-5/BanglaSER/train_data.json",'r') as f:
    train_data = json.load(f)

with open("/content/t9h6p943xy-5/BanglaSER/test_data.json",'r') as f:
    test_data = json.load(f)

In [12]:

train_data

[{'id': '03-01-03-02-02-03-29',
  'emotion': 'Angry',
  'length': 3.4825208333333335,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 29/03-01-03-02-02-03-29.wav'},
 {'id': '03-01-03-02-01-02-22',
  'emotion': 'Angry',
  'length': 3.6824583333333334,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 22/03-01-03-02-01-02-22.wav'},
 {'id': '03-01-03-02-03-01-16',
  'emotion': 'Angry',
  'length': 3.583125,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 16/03-01-03-02-03-01-16.wav'},
 {'id': '03-01-01-01-02-02-31',
  'emotion': 'Happy',
  'length': 3.5330839002267576,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 31/03-01-01-01-02-02-31.wav'},
 {'id': '03-01-03-02-01-01-18',
  'emotion': 'Angry',
  'length': 3.8287291666666667,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 18/03-01-03-02-01-01-18.wav'},
 {'id': '03-01-01-01-03-02-18',
  'emotion': 'Happy',
  'length': 3.4660208333333333,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 18/03-01-01-01-03-02-18.wav'},
 {'id': '03-01-05-01

In [ ]:
test_data

[{'id': '03-01-01-01-02-03-10',
  'emotion': 'Happy',
  'length': 3.150166666666667,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 10/03-01-01-01-02-03-10.wav'},
 {'id': '03-01-02-01-02-03-24',
  'emotion': 'Sad',
  'length': 3.5460833333333333,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 24/03-01-02-01-02-03-24.wav'},
 {'id': '03-01-02-01-01-01-12',
  'emotion': 'Sad',
  'length': 3.2226666666666666,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 12/03-01-02-01-01-01-12.wav'},
 {'id': '03-01-02-01-01-02-02',
  'emotion': 'Sad',
  'length': 3.5342403628117913,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-02-01-01-02-02.wav'},
 {'id': '03-01-05-01-02-02-02',
  'emotion': 'Neutral',
  'length': 3.0736961451247167,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 02/03-01-05-01-02-02-02.wav'},
 {'id': '03-01-01-01-01-02-23',
  'emotion': 'Happy',
  'length': 3.2354195011337867,
  'wav': '/content/t9h6p943xy-5/BanglaSER/Actor 23/03-01-01-01-01-02-23.wav'},
 {'id': '03-01-

In [13]:

train_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(train_data)
]
train_dataset = Dataset.from_list(train_samples_list)

In [14]:
test_samples_list = [
    {**sample_data, "id": i} for i, sample_data in enumerate(test_data)
]
test_dataset = Dataset.from_list(test_samples_list)

In [17]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav'],
        num_rows: 233
    })
})

In [33]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [34]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

Map:   0%|          | 0/233 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 928
    })
    test: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
    dev: Dataset({
        features: ['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'],
        num_rows: 233
    })
})

In [36]:

dataset['train'][0].keys()

dict_keys(['id', 'emotion', 'length', 'wav', 'audio_array', 'sample_rate'])

In [39]:
audio_data = dataset['train'][0]['audio_array'] # Access data using the 'dataset' variable
print(audio_data)

[-2.5569461286067963e-05, 5.962028808426112e-06, 2.4385230062762275e-05, -4.8756730393506587e-05, 2.1314321202225983e-05, 2.303132350789383e-05, -2.2256464944803156e-05, -4.1134153434541076e-05, -3.9371872844640166e-05, -7.773708057356998e-05, -0.00017516492516733706, -0.0001979352382477373, -0.0001303247408941388, -0.00017197555280290544, -0.000176810979610309, -2.079261321341619e-05, -0.00010540234507061541, -0.00023007036361377686, -0.0002694410504773259, -0.00029518789961002767, -0.0003914957633242011, -0.0003640481154434383, -0.00031823557219468057, -0.0001886918325908482, -0.00010387235670350492, -0.00023204772151075304, -0.00015180502668954432, -0.00016755718388594687, -0.0001938247587531805, -0.00021058171114418656, -0.0002859779342543334, -0.00023219574359245598, -0.0002692517009563744, -0.0003565506194718182, -0.00022903220087755471, -8.565065218135715e-05, -0.00011678837472572923, -5.867362779099494e-05, 8.734119182918221e-05, 9.57904412643984e-05, 4.1022100049303845e-05, 1.

In [21]:

!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm


In [1]:
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset, DatasetDict, Audio
from transformers import Wav2Vec2FeatureExtractor, AutoModelForAudioClassification
from sklearn.metrics import recall_score, accuracy_score
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer

# ---- 1. Metric Functions ----
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# ---- 2. Collate Function for DataLoader ----
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    emotions = [int(example["emotion"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "emotion": torch.tensor(emotions)}

# ---- 3. Helper Functions for Labels ----
def to_label(emotion, label2id):
    return label2id.get(emotion, -1)  # Convert emotion to label ID

# ---- 4. Load and Prepare Dataset ----
def load_banglaser():
    # Load JSON files for train, test, and dev
    train_df = pd.read_json("/content/t9h6p943xy-5/BanglaSER/train_data.json")
    test_df = pd.read_json("/content/t9h6p943xy-5/BanglaSER/test_data.json")
    dev_df = pd.read_json("/content/t9h6p943xy-5/BanglaSER/test_data.json")

    # Define label mappings
    labels = ['Angry', 'Happy', 'Neutral', 'Sad']
    label2id = {label: idx for idx, label in enumerate(labels)}
    id2label = {idx: label for label, idx in label2id.items()}

    # Map emotion names to label IDs
    train_df["emotion"] = train_df["emotion"].apply(lambda x: to_label(x, label2id))
    test_df["emotion"] = test_df["emotion"].apply(lambda x: to_label(x, label2id))
    dev_df["emotion"] = dev_df["emotion"].apply(lambda x: to_label(x, label2id))

    # Convert DataFrames to Dataset objects
    train_audio_data = Dataset.from_pandas(train_df).cast_column("wav", Audio(sampling_rate=16_000))
    test_audio_data = Dataset.from_pandas(test_df).cast_column("wav", Audio(sampling_rate=16_000))
    dev_audio_data = Dataset.from_pandas(dev_df).cast_column("wav", Audio(sampling_rate=16_000))

    return label2id, id2label, DatasetDict({'train': train_audio_data, 'test': test_audio_data, 'dev': dev_audio_data})

# ---- 5. Preprocessing Function ----
def preprocess_function(examples, feature_extractor):
    audio_arrays = [x["wav"] for x in examples["wav"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, padding=True, truncation=True)
    return inputs
    return inputs
# ---- 6. Initialize Model ----
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(model_name, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.to(device)
    return model

# ---- 7. Training Function and Trainer Setup ----
def train_banglaser():
    # Load dataset and label mappings
    label2id, id2label, dataset_dict = load_banglaser()

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess data
    dataset_dict = dataset_dict.map(lambda x: preprocess_function(x, feature_extractor), batched=True)

    # Initialize model
    model = model_ft(label2id, id2label, num_classes=len(label2id))

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,  # Updated to 10 epochs
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
    )

    # Custom metric computation
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = torch.argmax(logits, dim=-1)
        return {
            "unweighted_recall": calculate_unweighted_recall(labels, predictions),
            "weighted_recall": calculate_weighted_recall(labels, predictions),
            "accuracy": calculate_accuracy(labels, predictions),
        }

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_dict['train'],
        eval_dataset=dataset_dict['test'],
        data_collator=collate_fn,
        tokenizer=feature_extractor,
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

# Run training
train_banglaser()


ModuleNotFoundError: No module named 'datasets'